# TENET: TExtual traiNing Examples from daTa

Google Colab pipeline to make TENET run. Just press 'Run all'. 

Experiment settings can be edited under Configurations. ```active_cfg``` contains the current configuration to be run.

## Setup



Run this section to install the project in the local Colab folder.

In [ ]:
%cd /content
%rm -rf eurecom-evidence-generator/
!git clone https://github.com/akatief/eurecom-evidence-generator.git
%cd eurecom-evidence-generator
!git checkout origin/develop

/content
Cloning into 'eurecom-evidence-generator'...
remote: Enumerating objects: 451, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 451 (delta 1), reused 7 (delta 1), pack-reused 427
Receiving objects: 100% (451/451), 131.61 KiB | 4.87 MiB/s, done.
Resolving deltas: 100% (245/245), done.
/content/eurecom-evidence-generator
Note: checking out 'origin/develop'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 2293580 Merge pull request #3 from akatief/NER-fix


In [ ]:
!pip install -r requirements.txt
!pip install --no-deps feverous

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 5.1 MB/s 
     |████████████████████████████████| 1.1 MB 5.1 MB/s 
     |████████████████████████████████| 147 kB 76.2 MB/s 
     |████████████████████████████████| 1.2 MB 56.2 MB/s 
     |████████████████████████████████| 4.9 MB 53.4 MB/s 
     |████████████████████████████████| 2.1 MB 46.5 MB/s 
     |████████████████████████████████| 149 kB 78.0 MB/s 
     |████████████████████████████████| 96 kB 7.5 MB/s 
     |████████████████████████████████| 175 kB 74.7 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 61 kB 8.4 MB/s 
     |████████████████████████████████| 98 kB 9.7 MB/s 
     |████████████████████████████████| 662 kB 66.7 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 895 kB 66.6 MB/s 
     |████████████████████████████████| 25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 144 kB 5.0 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Configurations

Customize your experiment by creating a new config. Note that you must have a valid FEVEROUS database and ToTTo model image on your Google Drive.

In [ ]:
class cfg:
  def __init__(self,data_path, model_path, positive_evidence, negative_evidence,
               wrong_cell, table_per_page, evidence_per_table, column_per_table,
               seed, strat):
    self.data_path = data_path
    self.model_path = model_path
    self.positive_evidence = positive_evidence
    self.negative_evidence = negative_evidence
    self.wrong_cell = wrong_cell
    self.table_per_page = table_per_page
    self.evidence_per_table = evidence_per_table
    self.column_per_table= column_per_table
    self.seed = seed
    self.strat = strat

# Set to your model path and desired configuration
cfg1 = cfg(data_path = '/content/drive/MyDrive/Datasets/filtereddb_st_2.db', model_path = '/content/drive/MyDrive/Colab Notebooks/exported_totto_large/1648208035/',
           positive_evidence = 4000, negative_evidence = 0, wrong_cell = 0, table_per_page = 1,
           evidence_per_table = 1, column_per_table= 2, seed = 2022, strat = 'entity')

cfg2 = cfg(data_path = '/content/drive/MyDrive/Datasets/filtereddb_st_2.db', model_path = '/content/drive/MyDrive/Colab Notebooks/exported_totto_large/1648208035/',
           positive_evidence = 4000, negative_evidence = 0, wrong_cell = 0, table_per_page = 1,
           evidence_per_table = 1, column_per_table= 3, seed = 2022, strat = 'entity')

#cfg2 = ...

In [ ]:
active_cfg = cfg1

## Experiment

Finally, run the claim generation pipeline. 

In [ ]:
%cd /content/eurecom-evidence-generator/

import json
import hydra

# TODO: try to understand because if it is not used you get error
import tensorflow_text

from src.claim import TextualClaim
from src.claim import FeverousGenerator, ToTToGenerator

from src.pipeline import ClaimGeneratorPipeline
from src.evidence import FeverousRetrieverRandom

retriever = FeverousRetrieverRandom(p_dataset=active_cfg.data_path,
                                     num_positive=active_cfg.positive_evidence,
                                     num_negative=active_cfg.negative_evidence,
                                     wrong_cell=active_cfg.wrong_cell,
                                     table_per_page=active_cfg.table_per_page,
                                     evidence_per_table=active_cfg.evidence_per_table,
                                     column_per_table=active_cfg.column_per_table,
                                     seed=active_cfg.seed,
                                     key_strategy=active_cfg.strat,
                                     )

generator = ToTToGenerator(encoding='totto', model_path=active_cfg.model_path)


pipeline = ClaimGeneratorPipeline([retriever, generator])

claims = pipeline.generate()

json_evidence = TextualClaim.to_json(claims)

%cd /content
with open(f'./data_col_{active_cfg.column_per_table}_strategy_{active_cfg.strat}_positive_{active_cfg.positive_evidence}_negative_{active_cfg.negative_evidence}.json', 'w', encoding='utf-8') as f:
    json.dump(json_evidence, f, ensure_ascii=False, indent=4)



/content/eurecom-evidence-generator


Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


ERROR src.logger 2022-06-06 09:42:08,767 | utils.py:33 | b'got Error "TableExceptionType.NO_ENOUGH_COL" for wikipage "Asota fulvia"'
ERROR src.logger 2022-06-06 09:42:08,773 | utils.py:33 | b'got Error "TableExceptionType.NO_ENOUGH_TBL" for wikipage "_title"'
ERROR src.logger 2022-06-06 09:42:08,775 | utils.py:33 | b'got Error "TableExceptionType.NO_ENOUGH_COL" for wikipage "Engleria"'
ERROR src.logger 2022-06-06 09:42:08,778 | utils.py:33 | b'got Error "TableExceptionType.NO_ENOUGH_TBL" for wikipage "_title"'
ERROR src.logger 2022-06-06 09:42:08,785 | utils.py:33 | b'got Error "TableExceptionType.NO_ENOUGH_COL" for wikipage "Pacific Trim"'
ERROR src.logger 2022-06-06 09:42:08,786 | utils.py:33 | b'got Error "TableExceptionType.NO_ENOUGH_TBL" for wikipage "_title"'
ERROR src.logger 2022-06-06 09:42:08,791 | utils.py:33 | b'got Error "TableExceptionType.NO_ENOUGH_COL" for wikipage "Hymenonema"'
ERROR src.logger 2022-06-06 09:42:08,792 | utils.py:33 | b'got Error "TableExceptionType.NO_E

KeyboardInterrupt: ignored